In [ ]:
import nltk
nltk.download('punkt')
from nltk.stem.porter import PorterStemmer

import torch 
import torch.nn as nn
import numpy as np

from torch.utils.data import Dataset, DataLoader


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import numpy as np

In [ ]:
import json

In [ ]:
stemmer=PorterStemmer()


In [ ]:
def tokenize(sentence):
    """
    split sentence into array of words/tokens
    a token can be a word or punctuation character, or number
    """
    return nltk.word_tokenize(sentence)





In [ ]:
def stem_word(word):
  return stemmer.stem(word.lower()) 
  
 
      # its preferable to  do lowercase of wordsfirst, but I didnt do it  yet the results are same
                                #if passed a list as it is ,it returns string orthrows a error list has no attribute lower() , but  passed list itemsone by one  we get a list it is most preferred



In [ ]:
def bag_of_words(tokenized_sentence, words):
    """
    return bag of words array:
    1 for each known word that exists in the sentence, 0 otherwise
    example:
    sentence = ["hello", "how", "are", "you"]
    words = ["hi", "hello", "I", "you", "bye", "thank", "cool"]
    bog   = [  0 ,    1 ,    0 ,   1 ,    0 ,    0 ,      0]
    """
    # stem each word
    sentence_words = [stem_word(word) for word in tokenized_sentence]  #  stemming of pattern tokens 
    # initialize bag with 0 for each word
    bag = np.zeros(len(words), dtype=np.float32) #np,zeros create a numpy array of given length
    for idx, w in enumerate(words):  #  The enumerate() method adds a counter to an iterable and returns it (the enumerate object).
        if w in sentence_words: 
            bag[idx] = 1.0

    return bag # returns a numpy array , since modified from np.zeros

Loading Json file

In [ ]:
with open("intent.json","r") as f:  # If we dont use "with" keyword , then we have to close the file with file.close()  https://www.geeksforgeeks.org/with-statement-in-python/
  j_file=json.load(f)

In [ ]:
all_words=[]
tags=[]
xy=[]

TOKENISING Words

In [ ]:
for   intent in  j_file["intents"]:  #  https://linuxhint.com/search_json_python/     to extract , intent , tag , or anything you can do is file_object["tag_name"]
 # but i guess you have to remove encapsulation one by one 
  tag=intent["tag"]
  tags.append(tag)  # dont do tags=tags.append(tag ) will give Nonetype error
  for pattern in intent["patterns"]:
    w=tokenize(pattern)   
    all_words.extend(w)   # if you want to add multiple elements to the list, you can use extend method if you want to add one element , then add append.
    xy.append((w,tag))  # here append because , we are adding one element ie is a tuple

Removing punctuations and stemming

In [ ]:
ignore_pun=["?" , "," , "." , "!"]

In [ ]:
all_words=  [stem_word(i)  for i in all_words if  i not in ignore_pun]

In [ ]:
print(all_words)

['hi', 'hey', 'how', 'are', 'you', 'is', 'anyon', 'there', 'hello', 'good', 'day', 'bye', 'see', 'you', 'later', 'goodby', 'thank', 'thank', 'you', 'that', "'s", 'help', 'thank', "'s", 'a', 'lot', 'which', 'item', 'do', 'you', 'have', 'what', 'kind', 'of', 'item', 'are', 'there', 'what', 'do', 'you', 'sell', 'do', 'you', 'take', 'credit', 'card', 'do', 'you', 'accept', 'mastercard', 'can', 'i', 'pay', 'with', 'paypal', 'are', 'you', 'cash', 'onli', 'how', 'long', 'doe', 'deliveri', 'take', 'how', 'long', 'doe', 'ship', 'take', 'when', 'do', 'i', 'get', 'my', 'deliveri', 'tell', 'me', 'a', 'joke', 'tell', 'me', 'someth', 'funni', 'do', 'you', 'know', 'a', 'joke']


In [ ]:
all_words=sorted(set(all_words)) #removing duplicate values ;used to convert any of the iterable to sequence of iterable elements with distinct elements

In [ ]:
print(all_words)

["'s", 'a', 'accept', 'anyon', 'are', 'bye', 'can', 'card', 'cash', 'credit', 'day', 'deliveri', 'do', 'doe', 'funni', 'get', 'good', 'goodby', 'have', 'hello', 'help', 'hey', 'hi', 'how', 'i', 'is', 'item', 'joke', 'kind', 'know', 'later', 'long', 'lot', 'mastercard', 'me', 'my', 'of', 'onli', 'pay', 'paypal', 'see', 'sell', 'ship', 'someth', 'take', 'tell', 'thank', 'that', 'there', 'what', 'when', 'which', 'with', 'you']


In [ ]:
X_train = [] # bag of words 
y_train = []  # the tags, greetings. delivery , payment tags
for (pattern_sentence, tag) in xy:
    # X: bag of words for each pattern_sentence
    bag = bag_of_words(pattern_sentence, all_words)    # bag return np array of each pattern 
    X_train.append(bag)  # one pattern one row,second pattern second row and so on . 
    # y: PyTorch CrossEntropyLoss needs only class labels, not one-hot
    label = tags.index(tag)   # getting the numerical represetation of tags
    y_train.append(label)



In [ ]:
X_train = np.array(X_train)   # just X_train gives a  list with  numpy arrays as its items , so we have to convert it into np.array

In [ ]:
y_train = np.array(y_train)   # just y_train gives a  list with  numpy arrays as its items , so we have to convert it into np.array

In [ ]:
# Hyper-parameters 
num_epochs = 1000
batch_size = 8
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)
print(input_size, output_size)

54 9


In [ ]:


class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples




In [ ]:
dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size) 
        self.l2 = nn.Linear(hidden_size, hidden_size) 
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        # no activation and no softmax at the end
        return out

In [ ]:
  





# how to decide the input output , hideen size and the optimizatoin function  ???
# what is the output and output size .
# Hyper-parameters 
num_epochs = 1000
batch_size = 8
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)
print(input_size, output_size)

class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = NeuralNet(input_size, hidden_size, output_size).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)
        
        # Forward pass
        outputs = model(words)
        # if y would be one-hot, we must apply
        # labels = torch.max(labels, 1)[1]
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch+1) % 100 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

54 9
Epoch [100/1000], Loss: 0.6378
Epoch [200/1000], Loss: 0.2328
Epoch [300/1000], Loss: 0.0186
Epoch [400/1000], Loss: 0.0066
Epoch [500/1000], Loss: 0.0071
Epoch [600/1000], Loss: 0.0042
Epoch [700/1000], Loss: 0.0020
Epoch [800/1000], Loss: 0.0012
Epoch [900/1000], Loss: 0.0007
Epoch [1000/1000], Loss: 0.0008
